<a href="https://colab.research.google.com/github/1900690/kyouyu/blob/main/%E3%82%A2%E3%83%97%E3%83%AA%E3%82%B1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E5%8C%96.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#streamlitの参考情報
[Streamlit API reference](https://docs.streamlit.io/library/api-reference)

[Streamlit configuration](https://docs.streamlit.io/library/advanced-features/configuration)

#pipインストールがいらないアプリを作る場合

In [1]:
#streamlitをインストール
!pip -q install streamlit

     |████████████████████████████████| 10.1 MB 3.7 MB/s 
     |████████████████████████████████| 111 kB 44.2 MB/s 
     |████████████████████████████████| 4.3 MB 29.6 MB/s 
     |████████████████████████████████| 181 kB 54.3 MB/s 
     |████████████████████████████████| 77 kB 2.2 MB/s 
     |████████████████████████████████| 164 kB 34.2 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 131 kB 42.2 MB/s 
     |████████████████████████████████| 428 kB 36.1 MB/s 
     |████████████████████████████████| 793 kB 36.6 MB/s 
     |████████████████████████████████| 130 kB 42.4 MB/s 
     |████████████████████████████████| 381 kB 46.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

content内にapp.pyファイルを作成し、その中にアプリを動かすコードを入力している。

コードの詳しい説明については"StreamlitアプリをGoogleColabから秒で外部公開する"(https://www.space-i.com/post-blog/googlecola%E4%B8%8A%E3%81%8B%E3%82%89streamlit%E3%82%A2%E3%83%97%E3%83%AA%E3%82%92-cloudflare%E7%B5%8C%E7%94%B1%E3%81%A7%E7%A7%92%E3%81%A7%E5%A4%96%E9%83%A8%E5%85%AC%E9%96%8B%E3%81%99%E3%82%8B/)

In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd
import streamlit as st
 
df = pd.DataFrame( np.random.randn(10, 4), columns=['a', 'b', 'c','d'] )
 
st.title("Simple Streamlit App")
 
st.write("📈　Line-Chart")
st.line_chart(df)
 
if st.checkbox('👓 Show DataFrame (with highlight_max)'):
 st.table(df.style.highlight_max(axis=0))

In [3]:
#app.pyの内容をタブで表示
from google.colab import files
files.view("/content")
files.view("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
#アクセス用ＵＲＬを取得する
#行コミ環境下ではＵＲＬを読み込むことができない
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501

2022-05-18 06:58:53.137 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.75.35.93:8501

npx: installed 22 in 1.67s
your url is: https://breezy-teeth-boil-34-75-35-93.loca.lt
  Stopping...
^C


上で表示されたＵＲＬをコピーして新しいタブで検索する

検索欄右端の”このページを共有”ボタンからＱＲコードを選択

作成したＱＲコードをスマホで読み取りスマホ上で確認する


#pipインストールが必要なアプリを作る場合
参考
Streamlit SharingとGitHubでStreamlitアプリを公開する方法https://sakizo-blog.com/524/

app.pyを記述し保存

In [23]:
%%writefile app.py

#エッジ検出を行うアプリ
import streamlit as st
from streamlit_webrtc import webrtc_streamer
import av
import cv2

st.title("動画からエッジ検出を行うアプリ")#タイトルを表示
st.write("アプリケーション化")#文章をページに記入する


class VideoProcessor:
  #スライダーの初期位置を定義
    def __init__(self) -> None:
        self.threshold1 = 100
        self.threshold2 = 200

  #recv()は画像フレームを受け取り、画像フレーム返す
    def recv(self, frame):
        img = frame.to_ndarray(format="bgr24")

        #ここに画像処理を行うコードを記述
        #cv2.Canny(img, 100, 200)はエッジ検出フィルタ
        #cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)はグレースケールをBGRに戻す
        img = cv2.cvtColor(cv2.Canny(img, self.threshold1, self.threshold2), cv2.COLOR_GRAY2BGR)

        return av.VideoFrame.from_ndarray(img, format="bgr24")

#webrtc_streamer:webブラウザを介した映像・音声の入出力を扱うコンポーネント
ctx = webrtc_streamer(    
    key="example",
    video_processor_factory=VideoProcessor,
    #サーバがローカルにない場合、映像・音声伝送の通信を確立するために設定
    rtc_configuration={
        "iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]
    }
)
if ctx.video_processor:
  #st.slider():スライダーを作る組み込みウィジェット
    ctx.video_processor.threshold1 = st.slider("Threshold1", min_value=0, max_value=1000, step=1, value=100)
    ctx.video_processor.threshold2 = st.slider("Threshold2", min_value=0, max_value=1000, step=1, value=200)

AttributeError: ignored

環境に必要な依存（streamlit-webrtcとopencv-python-headless）をインストールするためにrequirements.txt作成

In [26]:
fname = 'requirements.txt'   # 出力ファイル名
f = open(fname,'w')            # ファイルを開く
f.write('opencv-python-headless==4.5.4.60\n') # ここから出力処理 \nは改行コマンド
f.write('streamlit==1.2.0\n')
f.write('streamlit-webrtc==0.35.1')
f.close()                      # ファイルを閉じる

githubで新しいレポジトリを作成？しそこに作った二つのファイルと、readme.txtを作る

streamlitにアクセスし、ログイン、